In [2]:
import pandas as pd

reviews = pd.read_json('data/ys-reviews-with-categories.json')

reviews.head()

,text,rating,gmap_id,category
0,Helped me alot. If it was for her suggesting m...,5.0,0x809ad993cd15cc69:0x38291ec9a11f59ed,beauty
1,Always leave feeling refreshed...polite and re...,5.0,0x809ad993cd15cc69:0x38291ec9a11f59ed,beauty
2,Professional massage place. The ladies know w...,5.0,0x809ad993cd15cc69:0x38291ec9a11f59ed,beauty
3,This place was pleasing to me. Very nice staff...,5.0,0x809ad993cd15cc69:0x38291ec9a11f59ed,beauty
4,Tough lady loves great pressure massage. Woohoo.,5.0,0x809ad993cd15cc69:0x38291ec9a11f59ed,beauty


In [72]:
# sample bigram classification from medium website
import nltk
from nltk.classify import NaiveBayesClassifier
from nltk.util import ngrams

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from string import punctuation

stop_words = stopwords.words('english')
def filtered_tokens(text: str) :
    text = text.lower()
    tokens = word_tokenize(text)    
    tokens = [word for word in tokens if word not in stop_words and word not in punctuation]
    return tokens

# Function to extract bigrams from a text
def extract_bigrams(tokens):
    return list(ngrams(tokens, 2))

some = reviews.loc[(reviews.category == 'beauty')].sample(10000)

tokens = [filtered_tokens(t) for t in some.text.tolist()]

some.rating.tolist()
review_w_class = [(t, "positive" if r > 4 else "negative") for t, r in zip(tokens, some.rating.tolist())]

# Combine the reviews into a single dataset

# Split the data into training and testing sets
training_data = review_w_class[:int(0.8 * len(review_w_class))]
testing_data = review_w_class[int(0.8 * len(review_w_class)):]

# Extract bigrams from the reviews and create a feature set
features = [(dict(extract_bigrams(review)), label) for (review, label) in training_data]

# Train a Naive Bayes classifier on the feature set
classifier = NaiveBayesClassifier.train(features)

# Evaluate the classifier on the testing data
accuracy = nltk.classify.accuracy(classifier, [(dict(extract_bigrams(review)), label) for (review, label) in testing_data])
print("Accuracy:", accuracy)

# Predict the sentiment of a new review
review = "really good"
bigrams = extract_bigrams(filtered_tokens(review))
sentiment = classifier.classify(dict(bigrams))
print("Sentiment:", sentiment)

classifier.show_most_informative_features()

Accuracy: 0.559
Sentiment: positive
Most Informative Features
                   never = 'going'        negati : positi =     30.0 : 1.0
                      20 = 'minutes'      negati : positi =     28.9 : 1.0
                   waste = 'time'         negati : positi =     27.4 : 1.0
               somewhere = 'else'         negati : positi =     22.1 : 1.0
                    told = "n't"          negati : positi =     20.8 : 1.0
                 someone = 'else'         negati : positi =     20.1 : 1.0
                     bad = 'customer'     negati : positi =     17.8 : 1.0
                       ’ = 'even'         negati : positi =     17.5 : 1.0
                    said = '``'           negati : positi =     16.2 : 1.0
                    nice = 'store'        negati : positi =     15.9 : 1.0


In [36]:
review = "Great food and our waiter, she was a very friendly super awesome waiter."
from nltk.util import ngrams
import pandas as pd

def getNgrams(n):
    return pd.Series(ngrams(word_tokenize(review), n))

df = pd.DataFrame({
    "1-gram" : getNgrams(1),
    "2-gram" : getNgrams(2),
    "3-gram" : getNgrams(3),
    "4-gram" : getNgrams(4),
})
print(review)
df

Great food and our waiter, she was a very friendly super awesome waiter.


,1-gram,2-gram,3-gram,4-gram
0,"(Great,)","(Great, food)","(Great, food, and)","(Great, food, and, our)"
1,"(food,)","(food, and)","(food, and, our)","(food, and, our, waiter)"
2,"(and,)","(and, our)","(and, our, waiter)","(and, our, waiter, ,)"
3,"(our,)","(our, waiter)","(our, waiter, ,)","(our, waiter, ,, she)"
4,"(waiter,)","(waiter, ,)","(waiter, ,, she)","(waiter, ,, she, was)"
5,"(,,)","(,, she)","(,, she, was)","(,, she, was, a)"
6,"(she,)","(she, was)","(she, was, a)","(she, was, a, very)"
7,"(was,)","(was, a)","(was, a, very)","(was, a, very, friendly)"
8,"(a,)","(a, very)","(a, very, friendly)","(a, very, friendly, super)"
9,"(very,)","(very, friendly)","(very, friendly, super)","(very, friendly, super, awesome)"
